# DataSet Construction

## Scrapping Index Components From BlackRock ETFs (IVV)

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import json
"""
Historical Data of ETF Components Extracted from BlackRock Chile website. 
A Long History (from 2006Q3) is Publicly Available for:
+ SP500 (IVV), 
+ RUSSELL1000 (IWB), 
+ RUSSELL2000 (IWM), 
+ RUSSELL3000 (IWV).
Vanguard site (the largest Global issuer for passive indexing) does not record historical data of index (ETF) components,
Moreover the corresponding ETFs were issued much later than year 2K
"""
BLACKROCK_CL_URL_ROOT = "https://www.blackrock.com/cl/productos/"

def index_components_finder(ticker="IVV", path = "./data/pkl/"):
    etf_url = {        
        "IVV": "239726/ishares-core-sp-500-etf", # iShares Core S&P 500 ETF
        "IWB": "239707/ishares-russell-1000-etf", # iShares Russell 1000 ETF
        "IWM": "239710/ishares-russell-2000-etf", # iShares Russell 2000 ETF
        "IWV": "239714/ishares-russell-3000-etf", # iShares Russell 3000 ETF
    }

    url = BLACKROCK_CL_URL_ROOT + etf_url[ticker] + "#tabsAll"    
    # request page
    html = requests.get(url).content
    soup = BeautifulSoup(html)

    # find available dates
    holdings = soup.find("div", {"id": "holdings"})
    dates_div = holdings.find_all("div", "component-date-list")[1]
    dates_div.find_all("option")
    dates = [option.attrs["value"] for option in dates_div.find_all("option")]

    # download constituents for each date
    constituents = pd.Series()
    for date in dates:
        resp = requests.get(BLACKROCK_CL_URL_ROOT
        +etf_url[ticker]
        +f"/1506433277024.ajax?tab=all&fileType=json&asOfDate={date}").content[3:]
        tickers = json.loads(resp)
        tickers = [(arr[0], arr[1]) for arr in tickers['aaData']]
        date = datetime.strptime(date, "%Y%m%d")
        constituents[date] = tickers

    constituents = constituents.iloc[::-1] # reverse into cronlogical order
    # for pickle filename construction (serial date)
    last_date = constituents.index[-1].strftime("%Y%m%d")
    pklfile_fullpath = path + ticker + "_historical_components_" + last_date + ".pkl"
    constituents = constituents.to_frame(name="components")
    constituents.to_pickle(pklfile_fullpath)
    return constituents

In [20]:
index_components = []
tickers = ["IVV", "IWB", "IWM", "IWV"]
for ticker in tickers:
    idx_compos = index_components_finder(ticker=ticker)
    index_components.append(idx_compos)

/tmp/ipykernel_25994/2275880088.py:34: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  constituents = pd.Series()
/tmp/ipykernel_25994/2275880088.py:34: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  constituents = pd.Series()
/tmp/ipykernel_25994/2275880088.py:34: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  constituents = pd.Series()
/tmp/ipykernel_25994/2275880088.py:34: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  constituents = pd.Series()


In [22]:
index_components[-1]

,components
2006-09-29,"[(P5N994, Petrocorp Inc Escrow), (TBV, TIENS B..."
2006-10-31,"[(P5N994, Petrocorp Inc Escrow), (TBV, TIENS B..."
2006-11-30,"[(P5N994, Petrocorp Inc Escrow), (TBV, TIENS B..."
2006-12-29,"[(P5N994, Petrocorp Inc Escrow), (TLCP, TELECO..."
2007-01-31,"[(P5N994, Petrocorp Inc Escrow), (TLCP, TELECO..."
...,...
2022-03-31,"[(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A..."
2022-04-29,"[(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A..."
2022-05-31,"[(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A..."
2022-06-30,"[(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A..."
